# Inheritance vs Instantiation

In [1]:
from __future__ import annotations
from typing import Any

In [2]:
def visualise_type(arg1:Any, default:str=...) -> None:
	if arg1 is type:
		print(f'{arg1.__name__}')
		return

	try:
		name = arg1.__name__
	except AttributeError:
		name = default if default is not ... else arg1

	print(f'{name}  --t->  ', end='')
	visualise_type(type(arg1))

In [3]:
def visualise_base(arg1:Any, default:str=...) -> None:
	if arg1 is object:
		print(f'{arg1.__name__}')
		return
	
	try:
		name = arg1.__name__
	except AttributeError:
		name = default if default is not ... else arg1

	print(f'{name}  --b->  ', end='')
	visualise_base(arg1.__base__)

python3 rules:
- all classes are instances of `type`
- all classes inherit from `object`
- literally every<u>thing</u> is an `object`
- literally every<u>thing</u> has a `type`
- `type.__base__` is `object`
- `object.__base__` is `None`
- `type(object)` is `type`
- `type(type)` is `type`

## Example 1 - types vs bases

In [4]:
int             # instance of type
                # subclass of object

zero = int()    # instance of int

visualise_type(zero)
visualise_base(int)

0  --t->  int  --t->  type
int  --b->  object


In [5]:
def f(): ...
visualise_type(f)
visualise_base(type(f))

f  --t->  function  --t->  type
function  --b->  object


## Example 2.1 - instantiation

In [6]:
class Person: ...   # Person instantiates type
simon = Person()    # simon instantiates Person

visualise_type(simon, 'simon')
visualise_base(Person)

simon  --t->  Person  --t->  type
Person  --b->  object


the above is the same as below

In [7]:
Person = type('Person', (), {}) # Person instantiates type
simon = Person()                # simon instantiates Person

visualise_type(simon, 'simon')
visualise_base(Person)

simon  --t->  Person  --t->  type
Person  --b->  object


## Example 2.2 - inheritance

In [8]:
class Person: ...       # Person instantiates type
grace = Person()        # grace instantiates Person

class Boy(Person): ...  # Boy inherits from Person
simon = Boy()           # simon instantiates Boy

visualise_type(simon, 'simon')
visualise_type(grace, 'grace')
visualise_base(Boy)

simon  --t->  Boy  --t->  type
grace  --t->  Person  --t->  type
Boy  --b->  Person  --b->  object


the above is the same as below

In [9]:
Person = type('Person', (), {})
grace  = Person()   # grace instantiates Person

Boy = type('Boy', (Person,), {}) # Boy inherits from Person
simon = Boy()       # simon instantiates Boy

visualise_type(simon, 'simon')
visualise_type(grace, 'grace')
visualise_base(Boy)

simon  --t->  Boy  --t->  type
grace  --t->  Person  --t->  type
Boy  --b->  Person  --b->  object


# Example 3.1 - metaclasses

In [10]:
class meta1(type): ...
class Class1(metaclass=meta1): ...
instance = Class1()

visualise_type(instance, 'instance')
visualise_base(meta1)
visualise_base(Class1)

instance  --t->  Class1  --t->  meta1  --t->  type
meta1  --b->  type  --b->  object
Class1  --b->  object


the above is the same as below

In [11]:
meta1 = type('meta1', (type,), {})
Class1 = meta1('Class1', (), {})
instance = Class1()

visualise_type(instance, 'instance')
visualise_base(meta1)
visualise_base(Class1)

instance  --t->  Class1  --t->  meta1  --t->  type
meta1  --b->  type  --b->  object
Class1  --b->  object


an example from python's built-in `enum` library:

In [12]:
from enum import Enum, EnumMeta

visualise_type(Enum)
visualise_base(EnumMeta)
visualise_base(Enum)

Enum  --t->  EnumMeta  --t->  type
EnumMeta  --b->  type  --b->  object
Enum  --b->  object


## Example 3.2 - meta-metaclass

In [13]:
class meta1(type): ...
class meta2(type, metaclass=meta1): ...
class Class1(metaclass=meta2): ...
instance = Class1()

visualise_type(instance, 'instance')
visualise_base(meta1)
visualise_base(meta2)
visualise_base(Class1)

instance  --t->  Class1  --t->  meta2  --t->  meta1  --t->  type
meta1  --b->  type  --b->  object
meta2  --b->  type  --b->  object
Class1  --b->  object


the above is the same as below

In [14]:
meta1 = type('meta1', (type,), {})
meta2 = meta1('meta2', (type,), {})
Class1 = meta2('Class1', (), {})
instance = Class1() # type: ignore

visualise_type(instance, 'instance')
visualise_base(meta1)
visualise_base(meta2)
visualise_base(Class1)

instance  --t->  Class1  --t->  meta2  --t->  meta1  --t->  type
meta1  --b->  type  --b->  object
meta2  --b->  type  --b->  object
Class1  --b->  object


## Example 4 - lifehacks.metaclasses

In [15]:
from lifehacks.metaclasses import meta

@meta
class meta2(type): ...

@meta2
class Class1: ...

instance = Class1()

visualise_type(instance, 'instance')
visualise_base(meta)
visualise_base(meta2)
visualise_base(Class1)

instance  --t->  Class1  --t->  meta2  --t->  meta  --t->  meta  --t->  type
meta  --b->  type  --b->  object
meta2  --b->  type  --b->  object
Class1  --b->  object


In [16]:
from lifehacks.metaclasses import enum, meta

@enum
class Palette: ...

visualise_type(Palette)
visualise_base(meta)
visualise_base(enum)

Palette  --t->  enum  --t->  meta  --t->  meta  --t->  type
meta  --b->  type  --b->  object
enum  --b->  type  --b->  object


# .

.

.

.
